# Question 3

In [1]:
using Plots, DataFrames, CSV, GLM
using Optim, Distributions, Random, ForwardDiff
using LinearAlgebra, StatsFuns, FixedEffectModels

In [2]:
df = DataFrame(CSV.File("data/ps1_ex3.csv"));

# I need to get the 0 share in each market 
gdf = groupby(df, :market)
gdf = combine(gdf, :Shares => sum)
gdf[!,:shares_0] = 1 .- gdf[!,:Shares_sum];
df = leftjoin(df, gdf, on=:market);

# Will need ln(s_jt / s_0t) and ln(s_jt)
df[!,:log_sj_s0] = log.(df[!,:Shares]./df[!,:shares_0]);
df[!,:log_sj] = log.(df[!,:Shares]);

# Part 1: Estimating $(\alpha, \beta)$

We have the moment condition $E[\xi_{jt} | z_{jt}] = 0$, hence we can estimate the following with TSLS: 

$$ ln\left(\frac{s_{jt}}{s_{0t}}\right) = - \alpha p_{jt} + x_{jt}' \beta + \xi_{jt}$$

In [3]:
estimates = reg(df, @formula(log_sj_s0 ~ x + (Prices ~ z)))
α = -coef(estimates)[3];
β = coef(estimates)[2];
df[!,:ξ] = (df[!,:log_sj_s0] - predict(estimates,df)) .+ coef(estimates)[1];

# Part 2: Finding price elasticities

$$\eta_{jkt} = \frac{\partial s_{jt}}{\partial p_{kt}} \frac{p_{kt}}{s_{jt}} = \begin{cases}
        \alpha p_{jt} (1-s_{jt}) & j=k \\
        - \alpha p_{kt} s_{jt} & j\neq k
    \end{cases}$$

In [4]:
# reshape dataframe of prices and shares
share_array = Array(unstack(df, :market, :Product, :Shares)[!,2:7]);
price_array = Array(unstack(df, :market, :Product, :Prices)[!,2:7]);

In [5]:
# for each market, we should get a JxJ matrix 
T = size(share_array)[1]
J = size(share_array)[2]

η = zeros(T, J^2)

for t in 1:T
    iter = 1
    for j in 1:J
        for k in 1:J
            if j == k 
                η[t, iter] = α * price_array[t,j] * (1-share_array[t,k])
            else 
                η[t, iter] = -1 * α * price_array[t,k] * share_array[t,j]
            end
        iter += 1
        end
    end
end

# average across each market
η̄ = reshape(mean(η, dims=1), 6, 6)

6×6 Matrix{Float64}:
  1.24962   -0.317914  -0.141565  -0.139219  -0.13808   -0.142108
 -0.317425   1.25124   -0.141836  -0.13975   -0.138349  -0.142341
 -0.286686  -0.287691   1.28915   -0.126127  -0.124954  -0.128527
 -0.287309  -0.288337  -0.128271   1.29387   -0.125171  -0.128882
 -0.28665   -0.287474  -0.127996  -0.125908   1.29143   -0.128494
 -0.287242  -0.288019  -0.128341  -0.126219  -0.125208   1.2908

# Part 3: Recover marginal costs
Assuming that each product-market is a different firm, we can obtain the following from the FOC from the firm optimization:
$$ s_{jt}(p) + (p_{jt} - mc_{jt}) \frac{\partial s_{jt}}{\partial p_{kt}} = 0 $$
which implies:
$$ s_{jt}(p) \frac{1}{\frac{\partial s_{jt}}{\partial p_{kt}}} + p_{jt} = mc_{jt} $$

We can equivalently write this as:
$$ mc_{jt} = p_{jt} + \frac{1}{\alpha}\frac{1}{(1-s_{jt})}$$
which makes it clear that average marginal costs can be related to prices and shares directly from the data given an estimate of $\alpha$.

In [6]:
# own partial
#df[!,:own_partial] = α .* df[!,:Shares] .* (1 .- df[!,:Shares]);
#df[!,:mc] = df[!,:Shares] ./ df[!,:own_partial] .+ df[!,:Prices];

df[!,:invshares] = 1 ./ (1 .- df[!,:Shares]);
df[!,:mc] = df[!,:Prices] + (1/α) .* df[!,:invshares];

means = groupby(df, :Product)
means = combine(means, [:mc,:Prices,:invshares] .=> mean)

Row,Product,mc_mean,Prices_mean,invshares_mean
,Int64,Float64,Float64,Float64
1,1,6.05278,3.35995,1.25883
2,2,6.06317,3.36753,1.26015
3,3,5.38743,3.03306,1.10061
4,4,5.39064,3.03977,1.09898
5,5,5.37942,3.03103,1.09782
6,6,5.39361,3.03815,1.10112


# Part 4: Product 1 exits the market
I think the following works:

- The regression predicts $ln(S_j/S_0)$, where $S_0 = 1 - \sum_k S_k$. Note $\sum_j S_j / S_0 = \frac{1-S_0}{S_0}$:
$$ \exp\left(\ln\left(\frac{S_j}{S_0}\right)\right) = \frac{S_j}{S_0} \\  \frac{\frac{S_j}{S_0}}{ \sum_j \frac{S_j}{S_0}} =  \frac{S_j}{1-S_0} = \frac{S_j}{\sum_k S_k} $$

MM: To simulate $j = 1$ leaving the market, we solve for the market prices that minimize the firms' first-order conditions as the objective, subject to demand (expressed as shares) as estimated above. We can use the fact that $\frac{\partial s_{k}}{\partial p_{k}} = -\alpha s_k (1-s_k)$. We can then plug the prices into the share equation to get simulated market shares. That is:
$$ \min_{p_{j}} \sum_{k = 2}^5 (FOC_k)^2  = \sum_{k = 2}^5 \bigg[ s_k\big(1 + \alpha (p_{k} - c_{k})(1 - s_k)\big) \bigg]^2 $$
$$ \text{s.t. } s_{j} = \frac{\exp(-\alpha p_{j} + x_{j}'\beta + \xi_{j})}{1 + \sum_{k = 2}^5 \exp(-\alpha p_{k} + x_{k}'\beta + \xi_{k})}$$ 
The gradient of the objective can be expressed as:
$$ \nabla_t(p) =\left\{  2\alpha s_j \left( -\alpha s_j (1-s_j)^2 FOC_k + \sum_{k = 2}^5 FOC_k (FOC_k - \alpha (p_k - c_k) s_k^2 \right) \right\}_{j = 2}^{5}$$ 

In [94]:
function implied_shares(J,x,ξ,α,β,p) # calculate market demand given parameters, prices, and products
    # numerator
    u = zeros(J);
    @. u = exp(-α * p + x * β + ξ);
    
    # denominator
    u_sum = sum(u) + 1;
    
    # calculate shares
    s = u ./ u_sum;
    
    return s
end

function foc(J,α,s,c,p)
    # calculate firm FOCs
    foc = zeros(J)
    @. foc = s + (α * (p - c) * s * (1 - s)); 
    
    return foc
end

function objective(x,ξ,α,β,c,p) # minimize total error in firm pricing FOCs in a market
    # number of parameters
    J = size(p)
    
    s = implied_shares(J,x,ξ,α,β,p)
    firmfoc = foc(J,α,s,c,p)
    
    # objective
    F = sum(firmfoc.^2)
    
    return F
end

function gradient!(x,ξ,α,β,c,G,p) # calculate gradient of objective !! this isn't right, probably
    # number of parameters
    J = size(p)
    
    s = implied_shares(J,x,ξ,α,β,p)
    firmfoc = foc(J,α,s,c,p)
    
    # component calculations
    inner = zeros(J)
    @. inner = firmfoc * (firmfoc - α * (p - c) * s^2)
    inner_sum = sum(inner)
    
    # calculate gradient
    @. G = 2 * α * s * (inner_sum - α * s * (1-s)^2 * firmfoc)
    
    return G
end

f(p) = objective(x,ξ,α,β,c,p)
g!(G,p) = gradient(x,ξ,α,β,c,G,p)

g! (generic function with 2 methods)

In [153]:
# Define counterfactual
simulation = df[!,[:market, :Product, :x, :ξ, :mc]];
simulation = simulation[(df.Product .!= 1), :];

# Solve for prices, looping through each market
prices1 = [];
prices2 = [];

focs = [];
shares = [];

# @time for mkt in 1:20
#     sim_mkt = simulation[(simulation.market .== mkt), :];
    
#     x = Array(sim_mkt[!,:x])
#     ξ = Array(sim_mkt[!,:ξ])
#     c = Array(sim_mkt[!,:mc])
    
#     # start somewhere close to true prices
#     params0 = ones(size(sim_mkt)[1]) .* 3;
    
#     optimum1 = optimize(p -> objective(x,ξ,α,β,c,p), params0, Optim.LBFGS(), autodiff=:forward, Optim.Options(f_tol = 1e-32, iterations = 8000))
#     p_sim1 = optimum1.minimizer;
#     append!(prices1,p_sim1)
# end

@time for mkt in 1:1000
    sim_mkt = simulation[(simulation.market .== mkt), :];
    
    x = Array(sim_mkt[!,:x])
    ξ = Array(sim_mkt[!,:ξ])
    c = Array(sim_mkt[!,:mc])
    
    # start somewhere close to true prices
    params0 = ones(size(sim_mkt)[1]) .* 3.0;
    # params0 = df[(df.market .== mkt),:Prices] 
    
    optimum2 = optimize(p -> objective(x,ξ,α,β,c,p), params0, Optim.LBFGS(), Optim.Options(f_tol = 1e-32, iterations = 10000))
    p_sim2 = optimum2.minimizer;
    append!(prices2,p_sim2)
    
    o = objective(x,ξ,α,β,c,p_sim2)
    s = implied_shares(x,ξ,α,β,p_sim2)
    
    push!(focs,o)
    append!(shares,s)
end

  0.313956 seconds (1.79 M allocations: 131.360 MiB, 10.04% gc time, 34.04% compilation time)


In [154]:
# simulation = df[!,[:market, :Product, :x, :ξ, :mc, :Prices, :Shares]];
# focs = [];
# shares = [];
# gradient = [];
# gradient2 = [];
# prices = [];
# G(x,ξ,α,β,c,p) = ForwardDiff.gradient(p -> objective(x,ξ,α,β,c,p), p)

# for mkt in 1:5
#     sim_mkt = simulation[(simulation.market .== mkt), :]
#     x = Array(sim_mkt[!,:x])
#     ξ = Array(sim_mkt[!,:ξ])
#     c = Array(sim_mkt[!,:mc])

#     p = Array(sim_mkt[!,:Prices])

#     o = objective(x,ξ,α,β,c,p)
#     s = implied_shares(x,ξ,α,β,p)

#     grad = G(x,ξ,α,β,c,p)
#     grad2 = zeros(size(p))
#     g!(grad2,x,ξ,α,β,c,p)

#     push!(focs,o)
#     push!(shares,s .- sim_mkt[!,:Shares])
#     push!(gradient,grad)
#     push!(gradient2,grad2)
# end

# gradient .- gradient2

s = df[(df.market .<= 1000), :Shares]
p = df[(df.market .<= 1000), :Prices]
println(mean(abs.(prices2 .- p)))
println(mean(abs.(shares .- s)))

println(maximum(focs))

LoadError: DimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 5000 and 6000")

In [ ]:
df_noprod1 = filter(row -> (row.Product != 1),  df)
df_noprod1[!,:blah] = exp.(predict(estimates, df_noprod1) ./ df_noprod1[!,:shares_0]) ;
df_noprod1[!,[:market, :Product, :blah]]

In [ ]:
df_noprod1 = filter(row -> (row.Product != 1),  df)

iterate = function(tol, model, data, α)
    
    err = 1e10
    iter = 0
    while err > tol 
        
        old_price = data[!,:Prices]
        
        # predict s_jt Q: how do I recover s_j?
        data[!,:new_sjs0] = exp.(predict(model, data))
        gdf = groupby(data, :market)
        gdf = combine(data, :new_sjs0 => sum)
        gdf[!,:shares_0] = 1 .- gdf[!,:new_sjs0];

        # use profit maximization to get new prices
        data[!,:own_partial] = α .* data[!,:Shares] .* (1 .- data[!,:Shares])
        data[!,:Prices] = data[!,:Shares] ./ data[!,:own_partial] .- data[!,:mc] 
        
        err = maximum(abs.(old_price .- data[!,:Prices]))
        iter += 1
    end
    return data, iter, err
end

iterate(1e-10, estimates, df_noprod1, α)

In [ ]:
err = 1e10
df_noprod1 = filter(row -> (row.Product != 1),  df)
data = df_noprod1
tol = 1e-10
iter = 0
    while err > tol 
        
        old_price = data[!,:Prices]
        # predict s_jt Q: how do I recover s_j?
        data[!,:new_sj] = exp.(predict(estimates, data))
        #gdf = groupby(data, :market)
        #gdf = combine(data, :new_sjs0 => sum)
        #gdf[!,:shares_0] = 1 .- gdf[!,:new_sjs0];

        # use profit maximization to get new prices
        data[!,:own_partial] = α .* data[!,:new_sj] .* (1 .- data[!,:new_sj])
        data[!,:Prices] = data[!,:new_sj] ./ data[!,:own_partial] .- data[!,:mc] 
        #println(data[!,:Prices] )

        err = maximum(abs.(old_price .- data[!,:Prices]))
        iter+=1
        println(abs.(old_price .- data[!,:Prices]))
        println(iter)
    end